# Artist Data Base

In [6]:
import sqlite3
import pandas as pd


conn =  sqlite3.connect("artist.db")

c = conn.cursor

path = "../../Documents/Ticket Sales.xlsx"

socials = pd.read_excel(path, sheet_name="Socials")
events = pd.read_excel(path, sheet_name="Events")
embrace = pd.read_excel(path, sheet_name="Embrace")
ticketweb = pd.read_excel(path, sheet_name="TicketWeb")


# Read data from the 'events' dataframe and create a table in the SQLite database
events.to_sql('EVENTS', conn, if_exists='replace', index=False, dtype={
    'Artist': 'TEXT',
    'Date': 'DATETIME',
    'Venue': 'TEXT',
    'Location': 'TEXT',
    '# of comments': 'REAL',
    'Presale': 'DATETIME',
    'Sellout Date': 'TEXT',
    'Sections': 'REAL',
    'Min Cost': 'FLOAT',
    'Max Resell': 'FLOAT',
    'Notes': 'TEXT'
})
embrace.to_sql('EMBRACE', conn, if_exists='replace', index=False, dtype={
    'Artist': 'TEXT',
    'Venue': 'TEXT',
    'start_date': 'DATETIME',
    'end_date': 'DATETIME',
    'Price': 'FLOAT',
    'Interested': 'INTEGER',
    'Went': 'INTEGER'
})
# Read data from the 'ticketweb' dataframe and create a table in the SQLite database
ticketweb.to_sql('TICKETWEB', conn, if_exists='replace', index=False, dtype={
    'Artist': 'TEXT',
    'Date': 'DATETIME',
    'Tier': 'TEXT',
    'TicketWeb': 'FLOAT',
    'Venue': 'TEXT'
})
# Read data from the 'socials' dataframe and create a table in the SQLite database
socials.to_sql('SOCIALS', conn, if_exists='replace', index=False, dtype={
    'Artist': 'TEXT',
    'instagram_username': 'TEXT',
    'instagram_followers': 'INTEGER',
    'spotify_id': 'TEXT',
    'spotify_genre': 'TEXT',
    'spotify_followers': 'INTEGER',
    'spotify_popularity': 'INTEGER',
    'spotify_listeners': 'INTEGER',
    'twitter_username': 'TEXT',
    'twitter_followers': 'INTEGER',
    'stubhub_url': 'TEXT',
    'stubhub_favourites': 'INTEGER'
})



194

In [11]:

socials.query("Artist == 'Timmy Trumpet' & instagram_followers > 1000000")    

,Artist,instagram_username,instagram_followers,spotify_id,spotify_genre,spotify_followers,spotify_popularity,spotify_listeners,twitter_username,twitter_followers,stubhub_url,stubhub_favourites
46,Timmy Trumpet,timmytrumpet,2000000.0,0CbeG1224FS58EUx4tPevZ,australian dance,1200719,73,10800000,TimmyTrumpet,176200.0,/timmy-trumpet-tickets/performer/1522792,33.0


In [3]:
conn.commit()

In [4]:
conn.close()

In [13]:
import psycopg2
from psycopg2 import sql

# Connect to PostgreSQL server
conn_pg = psycopg2.connect(
    dbname="postgres",
    user="your_username",
    password="your_password",
    host="localhost",
    port="5432"
)
conn_pg.autocommit = True
cursor_pg = conn_pg.cursor()

# Create a new database
cursor_pg.execute("CREATE DATABASE artist_db")

# Close the initial connection and connect to the new database
cursor_pg.close()
conn_pg.close()

conn_pg = psycopg2.connect(
    dbname="artist_db",
    user="your_username",
    password="your_password",
    host="localhost",
    port="5432"
)
cursor_pg = conn_pg.cursor()

# Create tables and insert data
def create_table_and_insert_data(table_name, dataframe, conn, cursor):
    # Create table
    create_table_query = sql.SQL(
        "CREATE TABLE {} ({});"
    ).format(
        sql.Identifier(table_name),
        sql.SQL(', ').join(
            sql.SQL("{} {}").format(
                sql.Identifier(col),
                sql.SQL(dtype)
            ) for col, dtype in zip(dataframe.columns, dataframe.dtypes.replace({
                'object': 'TEXT',
                'int64': 'INTEGER',
                'float64': 'FLOAT',
                'datetime64[ns]': 'TIMESTAMP'
            }).items())
        )
    )
    cursor.execute(create_table_query)
    conn.commit()

    # Insert data
    for i, row in dataframe.iterrows():
        insert_query = sql.SQL(
            "INSERT INTO {} VALUES ({});"
        ).format(
            sql.Identifier(table_name),
            sql.SQL(', ').join(sql.Placeholder() * len(row))
        )
        cursor.execute(insert_query, tuple(row))
    conn.commit()

# Create tables and insert data for each dataframe
create_table_and_insert_data('events', events, conn_pg, cursor_pg)
create_table_and_insert_data('embrace', embrace, conn_pg, cursor_pg)
create_table_and_insert_data('ticketweb', ticketweb, conn_pg, cursor_pg)
create_table_and_insert_data('socials', socials, conn_pg, cursor_pg)

# Close the connection
cursor_pg.close()
conn_pg.close()

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
